In [ ]:
! wget https://transfer.sh/ttw1Qk/merged_clean.txt.zip

--2022-02-10 22:00:04--  https://transfer.sh/ttw1Qk/merged_clean.txt.zip
Resolving transfer.sh (transfer.sh)... 144.76.136.153
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4652905 (4.4M) [application/zip]
Saving to: ‘merged_clean.txt.zip’

merged_clean.txt.zi 100%[===================>]   4.44M  4.12MB/s    in 1.1s    

2022-02-10 22:00:06 (4.12 MB/s) - ‘merged_clean.txt.zip’ saved [4652905/4652905]



In [ ]:
! unzip merged_clean.txt.zip

Archive:  merged_clean.txt.zip
  inflating: merged_clean.txt        


In [ ]:
! pip install hmmlearn

In [ ]:
! head merged_clean.txt


Lovely Ilonka

There was once a king's son who told his father that he wished to marry.

'No, no!' said the king; 'you must not be in such a hurry. Wait till you
have done some great deed. My father did not let me marry till I had won
the golden sword you see me wear.'

The prince was much disappointed, but he never dreamed of disobeying his


# Hidden Markov Model

In [ ]:
import sys, json, codecs, pickle, argparse
import numpy as np
import string

import joblib
from sklearn.preprocessing import LabelEncoder
from hmmlearn import hmm
from nltk import FreqDist

In [ ]:
file = open("merged_clean.txt", 'r', encoding='utf-8').read()
sentences = file.split("\n")
table = str.maketrans('', '', string.punctuation)

for index, sentence in enumerate(sentences):
  if sentence == "":
      del sentences[index]

In [ ]:
lines = [line.split() for line in sentences]

for line_index, line in enumerate(lines):
  for word_index, word in enumerate(line):
    lines[line_index][word_index] = word.translate(table)

words = [word.lower() for line in lines for word in line]

table = str.maketrans('', '', string.punctuation)
for index, word in enumerate(words):
    words[index] = word.translate(table)

alphabet = set(words)
le = LabelEncoder()
le.fit(list(alphabet))

seq = le.transform(words)
features = np.fromiter(seq, np.int64)
features = features.reshape(-1, 1)
# features = np.atleast_2d(features).T
fd = FreqDist(seq)

In [ ]:
print(len(alphabet))
print(words[:10])
print(seq[:10])
print(features[:10])
print(fd.get(0))

39738
['lovely', 'ilonka', 'there', 'was', 'once', 'a', 'kings', 'son', 'who', 'told']
[21341 17993 35093 38019 24826   574 19881 32604 38667 35713]
[[21341]
 [17993]
 [35093]
 [38019]
 [24826]
 [  574]
 [19881]
 [32604]
 [38667]
 [35713]]
225


In [ ]:
model = hmm.MultinomialHMM(n_components=3, init_params="ste")

In [ ]:
print(lines[:2])
print(features[:2])

[['Lovely', 'Ilonka'], ['There', 'was', 'once', 'a', 'kings', 'son', 'who', 'told', 'his', 'father', 'that', 'he', 'wished', 'to', 'marry']]
[[21341]
 [17993]]


In [ ]:
model = model.fit(features)

In [ ]:
symbols, _states = model.sample(20, random_state=42)
output = le.inverse_transform(np.squeeze(symbols))
for word in output:
        print(word, end=" ")

who set called to the with did could land gist attend him stupid nearly always ceiling which the at man 

# LSTM

In [ ]:
import string
import numpy as np

In [ ]:
file = open("merged_clean.txt", 'r', encoding='utf-8').read()
sentences = file.split("\n")
table = str.maketrans('', '', string.punctuation)

for index, sentence in enumerate(sentences):
  if sentence == "":
      del sentences[index]

In [ ]:
size = int(len(sentences) / 9)
sentences = sentences[:size]

In [ ]:
lines = [line.split() for line in sentences]

for line_index, line in enumerate(lines):
  for word_index, word in enumerate(line):
    lines[line_index][word_index] = word.translate(table)

words = [word.lower() for line in lines for word in line]

In [ ]:
chars = sorted(list(set(words)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(words)
n_vocab = len(chars)
print(f"Total Characters: {n_chars}")
print(f"Total Vocab: {n_vocab}")

Total Characters: 271446
Total Vocab: 11982


In [ ]:
X = ["hello", "my", "name", "is", "fran"]
Y = []

In [ ]:
seq_length = 20

data_X = []
data_Y = []

for i in range(0, n_chars - seq_length, 1):
	seq_in = words[i:i + seq_length]
	seq_out = words[i + seq_length]
	data_X.append([char_to_int[char] for char in seq_in])
	data_Y.append(char_to_int[seq_out])

In [ ]:
print(data_X[0])
print(data_Y[0])

[6363, 5425, 10616, 11480, 7301, 130, 5949, 9753, 11674, 10770, 5189, 3994, 10572, 5015, 11755, 10761, 6524, 7106, 7106, 9025]
[5425, 10616, 11480, 7301, 130, 5949, 9753, 11674, 10770, 5189, 3994, 10572, 5015, 11755, 10761, 6524, 7106, 7106, 9025, 10579]


In [ ]:
n_patterns = len(data_X)
print(f"Total patterns: {n_patterns}")

Total patterns: 271426


In [ ]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
X = np.reshape(data_X, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)

In [ ]:
# one hot encode the output variable
# y = tf.keras.utils.to_categorical(data_Y)
# y = np_utils.to_categorical(data_Y)
y = np.fromiter(Y, np.int64).reshape(-1, 1)

In [ ]:
print(X.shape[1])
print(X.shape[2])
# print(len(data_Y))

20
1
271426


In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# fit the model
model.fit(data_X, data_Y, epochs=1, batch_size=64, callbacks=callbacks_list)

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
start = np.random.randint(0, len(X)-1)
pattern = data_X[start]

print(f"Seed: {' '.join([int_to_char[value] for value in pattern])}")
# generate characters
for i in range(20):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(f"{result} ")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("Done.")

# GPT-2

In [ ]:
! pip install --force-reinstall transformers[tf-cpu] tensorflow==2.7.0

     |████████████████████████████████| 3.5 MB 7.8 MB/s 
     |████████████████████████████████| 489.6 MB 23 kB/s 
     |████████████████████████████████| 4.1 MB 48.9 MB/s 
     |████████████████████████████████| 126 kB 65.1 MB/s 
     |████████████████████████████████| 1.1 MB 57.4 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 463 kB 59.8 MB/s 
     |████████████████████████████████| 5.8 MB 48.5 MB/s 
     |████████████████████████████████| 57 kB 6.1 MB/s 
     |████████████████████████████████| 1.3 MB 45.3 MB/s 
     |████████████████████████████████| 42 kB 841 kB/s 
     |████████████████████████████████| 14.5 MB 45.3 MB/s 
     |████████████████████████████████| 15.7 MB 54.1 MB/s 
     |████████████████████████████████| 4.1 MB 41.2 MB/s 
     |████████████████████████████████| 2.1 MB 46.7 MB/s 
     |████████████████████████████████| 65 kB 4.5 MB/s 
     |████████████████████████████████| 156 kB 72.0 MB/s 
     |███████████████

In [ ]:
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2") 

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
print('vocabulary size: %d, max squence length: %d' % (tokenizer.vocab_size, tokenizer.model_max_length))
print('tokenize sequence "Once upon a time in a little village":', tokenizer('Once upon a time in a little village'))

vocabulary size: 50257, max squence length: 1024
tokenize sequence "Once upon a time in a little village": {'input_ids': [7454, 2402, 257, 640, 287, 257, 1310, 7404], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="merged_clean.txt",
    block_size=128)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="merged_clean.txt",
    block_size=128)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
print(tokenizer.decode(train_dataset[5]))

till they reached a great stone wall, many, many feet high.

'Now, prince,' said the magpie, 'the three bulrushes are behind that
wall.'

The prince wasted no time. He set his horse at the wall and leaped over
it. Then he looked about for the three bulrushes, pulled them up and
set off with them on his way home. As he rode along one of the bulrushes
happened to knock against something. It split open and, only think! out
sprang a lovely girl, who said: 'My heart's


In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = 'data/out', # the output directory for the model predictions and checkpoints
    overwrite_output_dir = True, # overwrite the content of the output directory
    per_device_train_batch_size = 4, # the batch size for training
    per_device_eval_batch_size = 4, # the batch size for evaluation
    learning_rate = 5e-5, # defaults to 5e-5
    num_train_epochs = 1, # total number of training epochs to perform
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25362
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6341


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model()

Saving model checkpoint to data/out
Configuration saved in data/out/config.json
Model weights saved in data/out/pytorch_model.bin


In [ ]:
generator = pipeline('text-generation', tokenizer='gpt2', model='data/out')

loading configuration file data/out/config.json
Model config GPT2Config {
  "_name_or_path": "data/out",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "use_cache": true,
  "vocab_size": 50257
}

loading configur

In [ ]:
print(generator('There is a building', max_length=500)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


There is a building in the centre of town, and there is a woman in it who is
playing on a string that has been brought from the forest from the
tree below on another rainy day.

'Let us go into the town to talk to her,' says she; and from that day
we talked to a man who was a minister of music, who had many other
music talents, and who was all over the whole city, and everybody in
the town who could sing, and if he could sing himself, great things might
be done to me for my entertainment.'

'Come on,' says I; 'come on, my husband.'

I took my wife and carried her about the country; and after we were
travelling through the country, I heard some one say to another that she
would have rather died than live here, for it made him very angry at my
playing, for the music had a happy effect upon him in the morning.
'Oh! he has some kind of magic stone,' said he; and he was angry with
him also, for he heard from his neighbour in this theatre that the music in
the city's theatre was too long and

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")

model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")

https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpq85poz2_


Downloading:   0%|          | 0.00/115 [00:00<?, ?B/s]

storing https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/257f58cef29abe2d3f559ce937e1c22360462b97e6096362a7b4e3249e0fbf65.9fc0c0f00754f9b14f9e59f344d88bc631c9b4bd7eeb23abcaf4de04945114d5
creating metadata file for /root/.cache/huggingface/transformers/257f58cef29abe2d3f559ce937e1c22360462b97e6096362a7b4e3249e0fbf65.9fc0c0f00754f9b14f9e59f344d88bc631c9b4bd7eeb23abcaf4de04945114d5
https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpp9zrtqxk


Downloading:   0%|          | 0.00/914 [00:00<?, ?B/s]

storing https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/80f30f43ed5e5159d1924e70134443c131f059187f562435a148107dbf002fec.05892ce96bf9f5c8bfde40968186593b5d0feec27999103dbf7ea1d3ca7d11e1
creating metadata file for /root/.cache/huggingface/transformers/80f30f43ed5e5159d1924e70134443c131f059187f562435a148107dbf002fec.05892ce96bf9f5c8bfde40968186593b5d0feec27999103dbf7ea1d3ca7d11e1
loading configuration file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/80f30f43ed5e5159d1924e70134443c131f059187f562435a148107dbf002fec.05892ce96bf9f5c8bfde40968186593b5d0feec27999103dbf7ea1d3ca7d11e1
Model config GPT2Config {
  "_name_or_path": "DeepESP/gpt2-spanish",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointin

...

storing https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/fb80110197f1b5c12472df03799ca4826aecf5fc97f788664c17035ea4ea004e.356a983865c1a96482df9e7e88858dbaa53473df869b0c13eb3c1d3c83a5e1fd
creating metadata file for /root/.cache/huggingface/transformers/fb80110197f1b5c12472df03799ca4826aecf5fc97f788664c17035ea4ea004e.356a983865c1a96482df9e7e88858dbaa53473df869b0c13eb3c1d3c83a5e1fd
https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp9n8oxfwu


Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

storing https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/33f6ff51b9dee7e39734d410c27efdddda2751cd489da15522ac58d7a9dc32bf.0d87709f2545d45784d5ff8d35dc319de93e4087b3c680e0e20b305fb949d2a7
creating metadata file for /root/.cache/huggingface/transformers/33f6ff51b9dee7e39734d410c27efdddda2751cd489da15522ac58d7a9dc32bf.0d87709f2545d45784d5ff8d35dc319de93e4087b3c680e0e20b305fb949d2a7
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/2b38e375a606e5f8f63dbc97cf8579eda0be824f6e74ed001fb5e9d11fb80774.9654cf615b595dbd278d7f33c84b3f4565133f4c3874d094bf360e7ea285dfaf
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/fb80110197f1b5c12472df03799ca4826aecf5fc97f788664c17035ea4ea004e.356a983865c1a96482df9e7e88858dbaa53473df869b0c13eb3c1d3c83a5e1fd
loading fi

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

storing https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/90a26ac08cbade5ec286472021c6965825a1b312caa7e003f948f2bc66ce7822.e3fcaa3ba58207141a5bbed65da898cffdda13dfa1f48580351532ef31c0f3e1
creating metadata file for /root/.cache/huggingface/transformers/90a26ac08cbade5ec286472021c6965825a1b312caa7e003f948f2bc66ce7822.e3fcaa3ba58207141a5bbed65da898cffdda13dfa1f48580351532ef31c0f3e1
loading weights file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/90a26ac08cbade5ec286472021c6965825a1b312caa7e003f948f2bc66ce7822.e3fcaa3ba58207141a5bbed65da898cffdda13dfa1f48580351532ef31c0f3e1
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at DeepESP/gpt2-spanish.
If your task is similar to the task the model of the checkpoint was trained on, you ca

In [ ]:
generator = pipeline('text-generation', tokenizer='DeepESP/gpt2-spanish', model='DeepESP/gpt2-spanish')

loading configuration file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/80f30f43ed5e5159d1924e70134443c131f059187f562435a148107dbf002fec.05892ce96bf9f5c8bfde40968186593b5d0feec27999103dbf7ea1d3ca7d11e1
Model config GPT2Config {
  "_name_or_path": "DeepESP/gpt2-spanish",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "c

In [ ]:
print(generator('Había una vez', max_length=100)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Había una vez más a nuestra merced que nos encontramos y, como él, se lo ha ganado, ¿no? 

—Aprender a manejar a los demás —dice Eternal y, con voz grave, le responde: 

—Eternal, cuando te pones en tu lugar…

—¿En la suya? 

Fear la mandíbula: 

—Mea a todos, ¿cómo está el bebé? 

—Suave… Le han atado ya los pies
